In [10]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

In [16]:
def callF1API(url):
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    xml_data = response.text
    return xml_data

In [19]:
#RESULTS OF 2023
url = "http://ergast.com/api/f1/2023/circuits"
xml_data = callF1API(url)

# Parse the XML data
root = ET.fromstring(xml_data)
# Extract relevant information
data = []
for circuit in root.findall('.//{http://ergast.com/mrd/1.5}Circuit'):
    circuit_id = circuit.get('circuitId')
    name = circuit.find('{http://ergast.com/mrd/1.5}CircuitName').text
    locality = circuit.find('.//{http://ergast.com/mrd/1.5}Locality').text
    country = circuit.find('.//{http://ergast.com/mrd/1.5}Country').text
    data.append({'CircuitId': circuit_id, 'CircuitName': name, 'Locality': locality, 'Country': country})

# Create a DataFrame
df = pd.DataFrame(data)
df.to_csv('circuits.csv')


In [21]:
url = "http://ergast.com/api/f1/2023/driverStandings"
xml_data = callF1API(url)

# Parse the XML data
root = ET.fromstring(xml_data)

# Extract relevant information
data = []
for driver_standing in root.findall('.//{http://ergast.com/mrd/1.5}DriverStanding'):
    position = driver_standing.get('position')
    points = driver_standing.get('points')
    wins = driver_standing.get('wins')
    driver = driver_standing.find('.//{http://ergast.com/mrd/1.5}Driver')
    driver_id = driver.get('driverId')
    given_name = driver.find('{http://ergast.com/mrd/1.5}GivenName').text
    family_name = driver.find('{http://ergast.com/mrd/1.5}FamilyName').text
    nationality = driver.find('{http://ergast.com/mrd/1.5}Nationality').text

    data.append({'Position': position, 'Points': points, 'Wins': wins, 'DriverId': driver_id, 'GivenName': given_name, 'FamilyName': family_name, 'Nationality': nationality})

# Create a DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.to_csv('driver_standings.csv')

In [44]:
# Iterate 22 times
for race_number in range(1, 23):
    
    # Construct the API URL for each race
    url = f"https://ergast.com/api/f1/2023/{race_number}/results"
    
    # Call F1 API
    xml_data = callF1API(url)
    # Parse the XML data
    root = ET.fromstring(xml_data)

    # Extract relevant information
    data = []
    for result in root.findall('.//{http://ergast.com/mrd/1.5}Result'):
        position = result.get('position')
        position_text = result.get('positionText')
        points = result.get('points')

        driver = result.find('.//{http://ergast.com/mrd/1.5}Driver')
        driver_id = driver.get('driverId')
        given_name = driver.find('{http://ergast.com/mrd/1.5}GivenName').text
        family_name = driver.find('{http://ergast.com/mrd/1.5}FamilyName').text

        race_name = root.find('.//{http://ergast.com/mrd/1.5}RaceName').text


        data.append({'Position': position, 'PositionText': position_text, 'Points': points, 'DriverId': driver_id,
                     'GivenName': given_name, 'FamilyName': family_name})

    # Create a DataFrame for each race
    df_race = pd.DataFrame(data)

    #Add column Race to enumerate the races
    df_race['Race'] = race_number
    df_race['Race_name'] = race_name
    
    # Concatenate all DataFrames
    df_combined = pd.concat([df_combined,df_race],ignore_index=True)


# Save combined DataFrame as CSV
df_combined.to_csv('combined_race_results.csv', index=False)

<?xml version="1.0" encoding="utf-8"?>
<?xml-stylesheet type="text/xsl" href="/schemas/mrd-1.5.xsl"?>
<MRData xmlns="http://ergast.com/mrd/1.5" series="f1" url="http://ergast.com/api/f1/2023/1/results" limit="30" offset="0" total="20">
	<RaceTable season="2023" round="1">
		<Race season="2023" round="1" url="https://en.wikipedia.org/wiki/2023_Bahrain_Grand_Prix">
			<RaceName>Bahrain Grand Prix</RaceName>
			<Circuit circuitId="bahrain" url="http://en.wikipedia.org/wiki/Bahrain_International_Circuit">
				<CircuitName>Bahrain International Circuit</CircuitName>
				<Location lat="26.0325" long="50.5106">
					<Locality>Sakhir</Locality>
					<Country>Bahrain</Country>
				</Location>
			</Circuit>
			<Date>2023-03-05</Date>
			<Time>15:00:00Z</Time>
			<ResultsList>
				<Result number="1" position="1" positionText="1" points="25">
					<Driver driverId="max_verstappen" code="VER" url="http://en.wikipedia.org/wiki/Max_Verstappen">
						<PermanentNumber>33</PermanentNumber>
						<Given